In [1]:
!pip install segmentation-models-pytorch

     |████████████████████████████████| 87 kB 3.3 MB/s eta 0:00:011
  Using cached torchvision-0.10.0-cp38-cp38-manylinux1_x86_64.whl (22.1 MB)
     |████████████████████████████████| 376 kB 34.4 MB/s eta 0:00:01
  Using cached pretrainedmodels-0.7.4-py3-none-any.whl
  Using cached efficientnet_pytorch-0.6.3-py3-none-any.whl
  Using cached torch-1.9.0-cp38-cp38-manylinux1_x86_64.whl (831.4 MB)
  Using cached munch-2.5.0-py2.py3-none-any.whl (10 kB)
     |████████████████████████████████| 76 kB 6.9 MB/s  eta 0:00:01
  Using cached typing_extensions-3.10.0.0-py3-none-any.whl (26 kB)
  Using cached Pillow-8.3.1-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (3.0 MB)


In [25]:
# from google.colab import drive
# drive.mount('/gdrive')

In [2]:
!pip install matplotlib

In [3]:
import os
os.environ[ 'CUDA_VISIBLE_DEVICES'] = '1'

import numpy as np
import cv2
import matplotlib.pyplot as plt

In [4]:
DATA_DIR = '/gdrive/MyDrive/Henry_Lab/Cap08102020_seg/'
SAVE_DIR = '/playpen1/qiuyang/refined_segmentation/models/'
FINETUNE2_DIR = '/playpen1/qiuyang/refined_segmentation/models/final/'
# x_train_dir  = os.path.join(DATA_DIR, 'train')
# y_train_dir = os.path.join(DATA_DIR, 'trainannot')
# x_valid_dir = os.path.join(DATA_DIR, 'val')
# y_valid_dir = os.path.join(DATA_DIR, 'valannot')

x_smalltrain_dir = '/playpen1/qiuyang/refined_segmentation/pascal_image_1'
y_smalltrain_dir = '/playpen1/qiuyang/refined_segmentation/pascal_mask_1'
x_finetune_dir = '/playpen1/qiuyang/refined_segmentation/pascal_image_50'
y_finetune_dir = '/playpen1/qiuyang/refined_segmentation/refined_mask_50'

x_test_dir = '/playpen1/qiuyang/refined_segmentation/pascal_test_image_20'
y_test_dir = '/playpen1/qiuyang/refined_segmentation/refined_test_mask_20'
x_unannotated_dir = '/playpen1/qiuyang/refined_segmentation/unannotated_image_4'
y_unannotated_dir = '/playpen1/qiuyang/refined_segmentation/unannotated_image_4'
# x_finetune2_dir = '/playpen1/qiuyang/refined_segmentation/pascal_finetune2_image_4'
# y_finetune2_dir = '/playpen1/qiuyang/refined_segmentation/pascal_finetune2_mask_4'

In [29]:
ls -a /playpen1/qiuyang/refined_segmentation/refined_test_mask_20

./          F00045.png  F00125.png  F00205.png  F00285.png  F00365.png
../         F00065.png  F00145.png  F00225.png  F00305.png  F00385.png
F00005.png  F00085.png  F00165.png  F00245.png  F00325.png
F00025.png  F00105.png  F00185.png  F00265.png  F00345.png


In [30]:
rm /playpen1/qiuyang/refined_segmentation/refined_test_mask_20/.DS_Store

rm: cannot remove '/playpen1/qiuyang/refined_segmentation/refined_test_mask_20/.DS_Store': No such file or directory


In [31]:
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

def bbox2_padding(img, mask):
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    height = rmax - rmin + 1
    #print(f"height is {height}")
                      
    cmin, cmax = np.where(cols)[0][[0, -1]]
    width = cmax - cmin + 1
    #print(f"width is {width}")

    rcenter = int((rmin+rmax) / 2)
    ccenter = int((cmin+cmax) / 2)
    
    half_length = max(abs(rcenter - rmin), abs(ccenter-cmin))
    length = max(height, width)
    # initialize new_image and new_mask
    new_image = np.zeros((length, length, 3), dtype=np.int32)
    new_mask = np.zeros((length, length, mask.shape[2]), dtype=np.int32)
    new_center = half_length

    start_row = max(0,new_center-int(height/2))
    start_col = max(0, new_center-int(width/2))
    new_image[start_row:start_row+height, start_col:start_col+width] = img[rmin:rmax+1,cmin:cmax+1]
    new_mask[start_row:start_row+height, start_col:start_col+width] = mask[rmin:rmax+1,cmin:cmax+1]

    #new_rmin = rcenter - max_dist
    #new_rmax = rcenter + max_dist
    #new_cmin = ccenter - max_dist
    #new_cmax = ccenter + max_dist
    new_image = new_image.astype('uint8')
    new_mask = new_mask.astype('uint8')
    return new_image, new_mask

In [4]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir
import albumentations as albu

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-zw_k4p4l
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-zw_k4p4l


In [33]:
class Dataset(BaseDataset):
    """Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    CLASSES = ['sky', 'building', 'pole', 'road', 'pavement', 
               'tree', 'signsymbol', 'fence', 'car', 
               'pedestrian', 'bicyclist', 'unlabelled']
    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
            onlyresize=False,
            needBbox=False
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id.replace("img", "seg")) for image_id in self.ids]  

        self.augmentation = augmentation
        self.preprocessing = preprocessing
        self.needBbox = needBbox
        self.onlyresize = onlyresize
        self.resize = albu.Compose([albu.Resize(height=612, width=512)])
    def __getitem__(self, i):
        # read data
#         print(self.images_fps[i])
#         print(self.masks_fps[i])
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.masks_fps[i], 0)
        # unsqueeze to (512, 512, 1)
        mask = np.expand_dims(mask,-1)
            
        if self.needBbox:
          # cut out areas outside of bbox and pad with black pixels
          image, mask = bbox2_padding(image, mask)
          # resize to (512, 512) and change back type to int32
          sample = self.resize(image=image, mask=mask)
          image, mask = sample['image'], sample['mask']

        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        if self.onlyresize:
            sample = self.resize(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            mask = mask.squeeze(0)
        
        #print(f'image shape is{image.shape}')
        #print(f'mask shape is {mask.shape}')
            
        return image, mask
    def __len__(self):
        return len(self.ids)

In [34]:
def get_training_augmentation():
    train_transform = [

        albu.HorizontalFlip(p=0.5),
        albu.ShiftScaleRotate(scale_limit=0.01, rotate_limit=10, shift_limit=0.0625, p=1, border_mode=0),

        albu.PadIfNeeded(min_height=512, min_width=512, always_apply=True, border_mode=0),
        albu.RandomCrop(height=320, width=320, always_apply=True),
        albu.Resize(height=512, width=512)
        #albu.RandomResizedCrop(height=512, width=512, scale=(0.8, 0.8))
    ]
    return albu.Compose(train_transform)

def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')

def to_image_scale(x, **kwargs):
    x = x.astype(np.float32)
    x = x /255.0
    x = x - 0.5
    x = x * 2
    return x
def to_mask_scale(x, **kwargs):
    x  = x.astype(np.float32)
    x = x/255.0
    return x
def get_transposed():
    _transform = [
      albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

def get_preprocessing():
    _transform = [
      albu.Lambda(image=to_image_scale, mask=to_mask_scale),
      albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)



In [6]:
import torch
import numpy as np
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
import torch.nn as nn

### Human Parsing

In [36]:
!pip install ninja

In [37]:
ls /playpen1/qiuyang/Self-Correction-Human-Parsing/checkpoints

final.pth


In [38]:
cd /playpen1/qiuyang/Self-Correction-Human-Parsing

/playpen1/qiuyang/Self-Correction-Human-Parsing


In [42]:
!nvidia-smi

Tue Jul 20 03:44:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN RTX           On   | 00000000:05:00.0 Off |                  N/A |
| 91%   86C    P2    92W / 280W |  24213MiB / 24217MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           On   | 00000000:06:00.0 Off |                  N/A |
| 41%   

In [40]:
#torch.cuda.memory_summary(device=None, abbreviated=False)

In [45]:
import networks
#device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = [1224, 1024]
output_size = [612, 512]
num_classes = 7

pretrained_model = networks.init_model('resnet101', num_classes=num_classes, pretrained=None)
state_dict = torch.load('/playpen1/qiuyang/Self-Correction-Human-Parsing/checkpoints/final.pth')['state_dict']
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:]  # remove `module.`
    new_state_dict[name] = v
pretrained_model.load_state_dict(new_state_dict)
pretrained_model = pretrained_model.to(device)
pretrained_model.eval()
old_model = pretrained_model

cuda


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
# def forward(pretrained_model, input, input_size):
#   output = pretrained_model(input)
#   upsample = torch.nn.Upsample(size=input_size, mode='bilinear', align_corners=True)
#   upsample_output = upsample(output[0][-1])
#   pred = upsample_output.softmax(dim=1)
#   pred = pred.argmax(dim=1)
#   return (pred != 0) 
def forward(pretrained_model, input, input_size):
  output = pretrained_model(input)
  upsample = torch.nn.Upsample(size=input_size, mode='bilinear', align_corners=True)
  upsample_output = upsample(output[0][-1])
  logit0 = upsample_output[:, 0, :, :]
  logit1,_ = torch.max(upsample_output[:, 1:, :, :], 1)
  logit = torch.stack((logit0, logit1), 1)
  return logit

In [ ]:
smooth = 1.
def dice_loss(y_pred_batch,y_true_batch):
    batch_size = y_pred_batch.shape[0]
    loss = 0
    for i in range(batch_size):
      y_pred = y_pred_batch[i]
      y_true = y_true_batch[i]
      product  =  y_pred*y_true    
      sec_product = (1-y_pred)*(1-y_true)
      true_positive_intersection = np.sum(product)
      true_negative_intersection = np.sum(sec_product)
      coefficient = ((true_positive_intersection +smooth)/ (np.sum(y_pred)+np.sum(y_true) +smooth )) + ((true_negative_intersection +smooth)/ (np.sum(1-y_pred)+np.sum(1-y_true) +smooth ))
      loss =  loss + (1-coefficient)
    return loss

In [ ]:
# #dataset_settings
# input_size = [1224, 1024]
# output_size = [612, 512]
# num_classes = 7
# label = ['Background', 'Head', 'Torso', 'Upper Arms', 'Lower Arms', 'Upper Legs', 'Lower Legs']

# #dataset
# dataset = Dataset(
#     x_test_dir, 
#     y_test_dir, 
#     preprocessing=get_preprocessing(),
#     onlyresize=True

# )
# batch_size = 4
# dataloader = DataLoader(dataset, batch_size=batch_size)
# loss_Softmax = nn.CrossEntropyLoss()

# # check performance of original model
# with torch.no_grad():
#     running_loss = 0
#     for batch_idx, (input, target) in enumerate(dataloader):
#         # input shape: [4, 3, 512, 512]
#         # target shape: [4, 512, 512]
#         input, target = input.cuda(), target.to(device, dtype=torch.int64)

#         #pred = forward(pretrained_model, input, input_size)
#         pred = forward(pretrained_model, input, output_size)
#         loss = loss_Softmax(pred, target)
#         running_loss += loss.item()
#         #print('batch_idx: %d loss: %.3f' % (batch_idx + 1, running_loss / (batch_idx + 1)))
#     print('Test set: batch_idx: %d loss: %.3f' % (batch_idx + 1, running_loss / (batch_idx + 1)))


In [ ]:
# Some Visualization Before Training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_dataset = Dataset(
    x_test_dir, 
    y_test_dir,  
    preprocessing=get_preprocessing(),
    onlyresize=True
)

test_dataset_vis = Dataset(
    x_test_dir, y_test_dir, 
    onlyresize=True
)
pretrained_model.eval()
# n is the pic we choose
n = 2
image_vis = test_dataset_vis[n][0].astype('uint8')
image, gt_mask = test_dataset[n]

gt_mask = gt_mask.squeeze()

x_tensor = torch.from_numpy(image).to(device).unsqueeze(0)
#pr_mask =  model.predict(x_tensor)
#pr_mask =  pretrained_model.predict(x_tensor)
pr_mask = forward(pretrained_model, x_tensor, output_size)
print(pr_mask.shape)
pred = pr_mask.softmax(dim=1).argmax(dim=1).squeeze(0).cpu().detach().numpy()


visualize(
  image=image_vis,
  ground_truth_mask=gt_mask, 
  seg=pred
)

In [ ]:
# # overfitting
# from torch.optim.lr_scheduler import StepLR
# input_size = [1224, 1024]
# output_size = [612, 512]
# num_classes = 7
# label = ['Background', 'Head', 'Torso', 'Upper Arms', 'Lower Arms', 'Upper Legs', 'Lower Legs']

# train_small_dataset = Dataset(
#     x_smalltrain_dir, 
#     y_smalltrain_dir,
#     preprocessing = get_preprocessing(),
#     onlyresize=True
# )

# dataset = Dataset(
#     x_test_dir, 
#     y_test_dir, 
#     preprocessing=get_preprocessing(),
#     onlyresize=True

# )
# batch_size = 4
# train_dataloader = DataLoader(train_small_dataset, batch_size=batch_size)
# dataloader = DataLoader(dataset, batch_size=batch_size)

# loss_Softmax = nn.CrossEntropyLoss(ignore_index=255)
# optimizer = torch.optim.Adam(pretrained_model.parameters(), lr=1e-5)
# scheduler = StepLR(optimizer, step_size=1, gamma=0.99)
# max_score = 0
# best_epoch = 0
# best_iou = 0
# training_loss = []
# testing_loss = []

# # def train
# def train(model, device, train_loader, optimizer, epoch):
#     model.train()
#     running_loss = 0
#     iou = 0
#     for batch_idx, (input, target) in enumerate(train_loader):
#         optimizer.zero_grad()
#         input, target = input.cuda(), target.to(device, dtype=torch.int64)
#         pr_mask = forward(model, input, output_size)
#         loss = loss_Softmax(pr_mask, target)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#         pred = pr_mask.softmax(dim=1).argmax(dim=1).squeeze(0).cpu().detach().numpy()
#         label = target.cpu().numpy()
#         #print(f'label.shape is {label.shape}')
#         iou += ((pred & label).sum((1, 2)).sum() + 1e-6) / ((pred | label).sum((1, 2)).sum() + 1e-6) 
        
#         if batch_idx == 0 and epoch % 5 == 0:
#             print('Epoch %d  visualization' % epoch + 1)
#             visualize(img=label[0], img2=pred[0])
    
#     # print every epoch
#     print('Epoch [%d] Training loss: %.3f' % (epoch + 1, running_loss / (batch_idx + 1)))
# #     print('[%d, %5d] loss: %.3f' %
# #         (epoch, batch_idx + 1, running_loss / (batch_idx + 1)))
#     print('Training set: IOU: {:.4f}'.format(
#         iou / len(train_dataloader.dataset) * batch_size
#     ))
#     epoch_loss = running_loss / (batch_idx + 1)
#     return iou, epoch_loss

# def test(model, device, dataloader, epoch):
#     with torch.no_grad():
#         running_loss = 0
#         for batch_idx, (input, target) in enumerate(dataloader):
#             input, target = input.cuda(), target.to(device, dtype=torch.int64)
#             pred = forward(model, input, output_size)
#             loss = loss_Softmax(pred, target)
#             running_loss += loss.item()
            
#         print('Testing loss: %.3f' % (running_loss / (batch_idx + 1)))
# #         print('Test set: batch_idx: %d loss: %.3f' % (batch_idx + 1, running_loss / (batch_idx + 1)))    
#     epoch_loss = running_loss / (batch_idx + 1)
#     return epoch_loss

# # set up epoches
# epoches = np.arange(0,100)
# # start training
# for i in epoches: 
#     iou, train_loss = train(pretrained_model, device, train_dataloader, optimizer, i)
#     test_loss = test(pretrained_model, device, dataloader, i)
#     training_loss.append(train_loss)
#     testing_loss.append(test_loss)


In [ ]:
# plt.plot(epoches, training_loss, 'b--', testing_loss, 'r--')

In [ ]:
# def train
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    running_loss = 0
    iou = 0
    for batch_idx, (input, target) in enumerate(train_loader):
        optimizer.zero_grad()
        input, target = input.cuda(), target.to(device, dtype=torch.int64)
        pr_mask = forward(model, input, output_size)
        loss = loss_Softmax(pr_mask, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        pred = pr_mask.softmax(dim=1).argmax(dim=1).squeeze(0).cpu().detach().numpy()
        label = target.cpu().numpy()
        iou += ((pred & label).sum((1, 2)).sum() + 1e-6) / ((pred | label).sum((1, 2)).sum() + 1e-6) 
        
        if batch_idx == 0 and epoch % 5 == 0:
            print('Epoch %d  visualization' % (epoch + 1))
            visualize(img=label[0], img2=pred[0])
            

    print('Epoch [%d] Training loss: %.3f' % (epoch + 1, running_loss / (batch_idx + 1)))
    print('Training set: IOU: {:.4f}'.format(
        iou / len(train_loader.dataset) * batch_size
    ))
    epoch_loss = running_loss / (batch_idx + 1)
    return iou, epoch_loss

def test(model, device, dataloader, epoch):
    with torch.no_grad():
        running_loss = 0
        for batch_idx, (input, target) in enumerate(dataloader):
            input, target = input.cuda(), target.to(device, dtype=torch.int64)
            pred = forward(model, input, output_size)
            loss = loss_Softmax(pred, target)
            running_loss += loss.item()
        print('Testing loss: %.3f' % (running_loss / (batch_idx + 1)))
    epoch_loss = running_loss / (batch_idx + 1)
    return epoch_loss

In [ ]:
ls -a /playpen1/qiuyang/refined_segmentation/pascal_image_50/

In [ ]:
rm -rf /playpen1/qiuyang/refined_segmentation/pascal_image_50/.ipynb_checkpoints

In [ ]:
from torch.optim.lr_scheduler import StepLR
#dataset_settings
input_size = [1224, 1024]
output_size = [612, 512]
num_classes = 7
label = ['Background', 'Head', 'Torso', 'Upper Arms', 'Lower Arms', 'Upper Legs', 'Lower Legs']

# set up testing dataset and training dataset
#dataset
train_finetune_dataset = Dataset(
    x_finetune_dir, 
    y_finetune_dir,
    preprocessing = get_preprocessing(),
    onlyresize=True
)


dataset = Dataset(
    x_test_dir, 
    y_test_dir, 
    preprocessing=get_preprocessing(),
    onlyresize=True

)

#initialize train dataloader
batch_size = 4
train_dataloader = DataLoader(train_finetune_dataset, batch_size=batch_size)

#initialize test dataloader
batch_size = 4
dataloader = DataLoader(dataset, batch_size=batch_size)

# initialize parameters
loss_Softmax = nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.Adam(pretrained_model.parameters(), lr=1e-5)
scheduler = StepLR(optimizer, step_size=1, gamma=0.99)
max_score = 0
best_epoch = 0
best_iou = 0
training_loss = []
testing_loss = []


# set up epoches
# epoches = np.arange(0,100)
epoches = np.arange(0,50)
# start training
for i in epoches: 
    iou, train_loss = train(pretrained_model, device, train_dataloader, optimizer, i)
    test_loss = test(pretrained_model, device, dataloader, i)
    training_loss.append(train_loss)
    testing_loss.append(test_loss)
    if iou > best_iou:
      best_epoch = i+1
      best_iou = iou
      torch.save(pretrained_model, os.path.join(SAVE_DIR, 'best_model%03d.pth'% (i+1)))
      print(f'Model saved epoch %03d' % (i+1))
    scheduler.step()

In [ ]:
plt.plot(epoches, training_loss, 'b--', testing_loss, 'r--')

In [ ]:
# Some visualization using the finetuned model

# load  model
#loaded_model = networks.init_model('resnet101', num_classes=num_classes, pretrained=None)
#loaded_model.load_state_dict(torch.load(os.path.join(SAVE_DIR, 'best_model87.pth')))
loaded_model = torch.load(os.path.join(SAVE_DIR, 'best_model018.pth'))
loaded_model.eval()
old_model.eval()

# train dataset sample
train_dataset = Dataset(
    x_finetune_dir, 
    y_finetune_dir,
    preprocessing = get_preprocessing(),
    onlyresize=True
)

train_dataset_vis = Dataset(
    x_finetune_dir, 
    y_finetune_dir, 
    onlyresize=True
)

print("Training Set:")
# n is the pic we choose
for n in range(2,4):
    image_vis = train_dataset_vis[n][0].astype('uint8')
    image, gt_mask = train_dataset[n]

    gt_mask = gt_mask.squeeze()

    x_tensor = torch.from_numpy(image).to(device).unsqueeze(0)
    #old
    old_pr_mask = forward(old_model, x_tensor,output_size)
    old_pred = old_pr_mask.softmax(dim=1).argmax(dim=1).squeeze(0).cpu().detach().numpy()
    old_mask = old_pred * 255
    #finetuned
    pr_mask = forward(loaded_model, x_tensor, output_size)
    pred = pr_mask.softmax(dim=1).argmax(dim=1).squeeze(0).cpu().detach().numpy()
    mask = pred * 255

    red_channel = image_vis[:,:,0] * 0.6 + mask * 0.4
    new_img  =  image_vis.copy()
    new_img[:,:,0] = red_channel

    
    visualize(
      image=image_vis,
      ground_truth_mask=gt_mask, 
      pretrained=old_pred,
      after_finetuned=pred,
      superimposed = new_img
    )
#----------------------------------------------------------------------------------#

# test dataset sample
test_dataset = Dataset(
    x_test_dir, 
    y_test_dir,  
    preprocessing=get_preprocessing(),
    onlyresize=True
)

test_dataset_vis = Dataset(
    x_test_dir, y_test_dir, 
    onlyresize=True
)

print("Testing Set:")
# n is the pic we choose
#n = 2
for n in range(2,4):
    image_vis = test_dataset_vis[n][0].astype('uint8')
    image, gt_mask = test_dataset[n]
    gt_mask  = gt_mask.squeeze()
    
    x_tensor = torch.from_numpy(image).to(device).unsqueeze(0)
    #old
    old_pr_mask = forward(old_model, x_tensor,output_size)
    old_pred = old_pr_mask.softmax(dim=1).argmax(dim=1).squeeze(0).cpu().detach().numpy()
    old_mask = old_pred * 255
    #finetuned
    pr_mask = forward(loaded_model, x_tensor, output_size)
    pred = pr_mask.softmax(dim=1).argmax(dim=1).squeeze(0).cpu().detach().numpy()
    mask = pred * 255

    red_channel = image_vis[:,:,0] * 0.6 + mask * 0.4
    new_img  =  image_vis.copy()
    new_img[:,:,0] = red_channel

    
    visualize(
      image=image_vis,
      ground_truth_mask=gt_mask,
      pretrained=old_pred,
      after_finetuned=pred,
      superimposed=new_img
    )


#----------------------------------------------------------------------------------#
# unannotated dataset sample
unannotated_dataset = Dataset(
    x_unannotated_dir, 
    y_unannotated_dir,  
    preprocessing=get_preprocessing(),
    onlyresize=True
)

unannotated_dataset_vis = Dataset(
    x_unannotated_dir, y_unannotated_dir, 
    onlyresize=True
)

print("Unannotated Set:")
# n is the pic we choose
#n = 2
for n in range(2,4):
    image_vis = unannotated_dataset_vis[n][0].astype('uint8')
    image, _ = unannotated_dataset[n]

    x_tensor = torch.from_numpy(image).to(device).unsqueeze(0)
    #old
    old_pr_mask = forward(old_model, x_tensor,output_size)
    old_pred = old_pr_mask.softmax(dim=1).argmax(dim=1).squeeze(0).cpu().detach().numpy()
    old_mask = old_pred * 255
    #finetuned
    pr_mask = forward(loaded_model, x_tensor, output_size)
    pred = pr_mask.softmax(dim=1).argmax(dim=1).squeeze(0).cpu().detach().numpy()
    mask = pred * 255

    red_channel = image_vis[:,:,0] * 0.6 + mask * 0.4
    new_img  =  image_vis.copy()
    new_img[:,:,0] = red_channel

    
    visualize(
      image=image_vis,
      pretrained=old_pred,
      after_finetuned=pred,
      superimposed=new_img
    )


In [ ]:
# loss on test set

# load model
loaded_model = torch.load(os.path.join(SAVE_DIR, 'best_model018.pth'))
loaded_model.eval() 
old_model.eval()

loss_Softmax = nn.CrossEntropyLoss(ignore_index=255)
# test dataset sample
test_dataset = Dataset(
    x_test_dir, 
    y_test_dir,  
    preprocessing=get_preprocessing(),
    onlyresize=True
)

test_dataset_vis = Dataset(
    x_test_dir, y_test_dir, 
    onlyresize=True
)

for n in range(len(test_dataset)):
    running_loss = 0
    
    image, gt_mask = test_dataset[n]
    image_vis = test_dataset_vis[n][0].astype('uint8')
    gt_mask = gt_mask.squeeze()
    gt_mask_tensor = torch.from_numpy(gt_mask).to(device, dtype=torch.int64).unsqueeze(0)
    x_tensor = torch.from_numpy(image).to(device).unsqueeze(0)
    
     #old
    old_pr_mask = forward(old_model, x_tensor,output_size)
    old_pred = old_pr_mask.softmax(dim=1).argmax(dim=1).squeeze(0).cpu().detach().numpy()
    old_mask = old_pred * 255
    
    #finetuned
    pr_mask = forward(loaded_model, x_tensor, output_size)
    
    # figure out loss
    #loss = loss_Softmax(pr_mask, target)
    loss = loss_Softmax(pr_mask, gt_mask_tensor)
    running_loss += loss.item()
    print('Testing loss: %.3f' % running_loss)
    
    pred = pr_mask.softmax(dim=1).argmax(dim=1).squeeze(0).cpu().detach().numpy()
    mask = pred * 255

    red_channel = image_vis[:,:,0] * 0.6 + mask * 0.4
    new_img  =  image_vis.copy()
    new_img[:,:,0] = red_channel

    
    visualize(
      image=image_vis,
      ground_truth_mask=gt_mask, 
      pretrained=old_pred,
      after_finetuned=pred,
      superimposed=new_img
    )
    



In [ ]:
# # further finetune on the four samples from test set
# finetune2_dataset = Dataset(
#     x_finetune2_dir, 
#     y_finetune2_dir,  
#     preprocessing=get_preprocessing(),
#     onlyresize=True
# )

# batch_size = 4
# finetune2_dataloader = DataLoader(finetune2_dataset, batch_size=batch_size)
# finetune2_loss_all = []
# epoches = np.arange(0, 5)
# best_iou = 0
# loss_Softmax = nn.CrossEntropyLoss(ignore_index=255)
# optimizer = torch.optim.Adam(loaded_model.parameters(), lr=1e-5)

# for i in epoches:
#     iou, finetune2_loss = train(loaded_model, device, finetune2_dataloader, optimizer, i)
#     finetune2_loss_all.append(finetune2_loss)
#     if iou > best_iou:
#       best_epoch = i+1
#       best_iou = iou
#       torch.save(loaded_model, os.path.join(FINETUNE2_DIR, 'best_model.pth'))
#       print(f'Model saved epoch {i+1}')

In [ ]:
# July 5 Generate masks using old model
# load old model
old_model.eval()

# set up save dir
fn2 = r'/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F%05d.png'
fn3 = r'/playpen1/qiuyang/refined_segmentation/pascal_mask_1294/F%05d.png'
fn4 = r'/playpen1/qiuyang/refined_segmentation/pascal_old_superimposed_1294/F%05d.png'

for fid in range(0, 1295):
    print(fn2%fid)
    image_vis = cv2.imread(fn2%fid)
    image = cv2.cvtColor(image_vis, cv2.COLOR_BGR2RGB)
    resize = albu.Compose([albu.Resize(height=612, width=512)])
    sample = resize(image=image)
    image = sample['image']
    preprocessing = get_preprocessing()
    sample = preprocessing(image=image)
    image = sample['image']
    x_tensor = torch.from_numpy(image).to(device).unsqueeze(0)
    pr_mask = forward(old_model, x_tensor, output_size)
    pred = pr_mask.softmax(dim=1).argmax(dim=1).squeeze(0).cpu().detach().numpy()
    mask = pred * 255
    
    #superimposed
    sample = resize(image=image_vis)
    image_vis = sample['image']
    image_vis[:,:,2] = image_vis[:,:,2] * 0.6 + mask * 0.4
    cv2.imwrite(fn4%fid, image_vis)


In [ ]:
# July 5 Generate superimposed masks
# load finetuned model
loaded_model = torch.load(os.path.join(SAVE_DIR, 'best_model018.pth'))
# loaded_model = torch.load(os.path.join(FINETUNE2_DIR, 'best_model.pth'))
loaded_model.eval() 

x_pascal1200_dir = '/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/'
y_pascal1200_dir = '/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/'

# set up save dir
fn2 = r'/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F%05d.png'
fn3 = r'/playpen1/qiuyang/refined_segmentation/finetuned_mask_1294/F%05d.png'
fn4 = r'/playpen1/qiuyang/refined_segmentation/finetuned_superimposed_1294/F%05d.png'
for fid in range(0, 1295):
    print(fn2%fid)
    image_vis = cv2.imread(fn2%fid)
    image = cv2.cvtColor(image_vis, cv2.COLOR_BGR2RGB)
    resize = albu.Compose([albu.Resize(height=612, width=512)])
    sample = resize(image=image)
    image = sample['image']
    preprocessing = get_preprocessing()
    sample = preprocessing(image=image)
    image = sample['image']
    x_tensor = torch.from_numpy(image).to(device).unsqueeze(0)
    pr_mask = forward(loaded_model, x_tensor, output_size)
    pred = pr_mask.softmax(dim=1).argmax(dim=1).squeeze(0).cpu().detach().numpy()
    mask = pred * 255
    
    #superimposed
    sample = resize(image=image_vis)
    image_vis = sample['image']
    image_vis[:,:,2] = image_vis[:,:,2] * 0.6 + mask * 0.4
#     visualize(
#         superimposed=image_vis,
#         img=mask,
        
#     )
    cv2.imwrite(fn4%fid, image_vis)

In [ ]:
# July 5 Generate masks
# load  model
loaded_model = torch.load(os.path.join(SAVE_DIR, 'best_model018.pth'))
# loaded_model = torch.load(os.path.join(FINETUNE2_DIR, 'best_model.pth'))
loaded_model.eval() 

# set up save dir
fn2 = r'/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F%05d.png'
fn3 = r'/playpen1/qiuyang/refined_segmentation/finetuned_mask_1294/F%05d.png'
for fid in range(0, 1295):
    print(fn2%fid)
    image = cv2.imread(fn2%fid)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resize = albu.Compose([albu.Resize(height=612, width=512)])
    sample = resize(image=image)
    image = sample['image']
    preprocessing = get_preprocessing()
    sample = preprocessing(image=image)
    image = sample['image']
    x_tensor = torch.from_numpy(image).to(device).unsqueeze(0)
    pr_mask = forward(loaded_model, x_tensor, output_size)
    pred = pr_mask.softmax(dim=1).argmax(dim=1).squeeze(0).cpu().detach().numpy()
    mask = pred * 255
#     visualize(
#         img=mask
#     )
    cv2.imwrite(fn3%fid, mask)

In [ ]:
# July 20 Generate selfie masks
# load  model
loaded_model = torch.load(os.path.join(SAVE_DIR, 'best_model018.pth'))
# loaded_model = torch.load(os.path.join(FINETUNE2_DIR, 'best_model.pth'))
loaded_model.eval() 

# set up save dir
fn2 = r'/playpen1/qiuyang/refined_segmentation/July_20_selfie/IMG_%04d.JPG'
fn3 = r'/playpen1/qiuyang/refined_segmentation/July_20_selfie/MASK_%04d.png'
for fid in range(3211, 3212):
    print(fn2%fid)
    image = cv2.imread(fn2%fid)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resize = albu.Compose([albu.Resize(height=612, width=512)])
    sample = resize(image=image)
    image = sample['image']
    preprocessing = get_preprocessing()
    sample = preprocessing(image=image)
    image = sample['image']
    x_tensor = torch.from_numpy(image).to(device).unsqueeze(0)
    pr_mask = forward(loaded_model, x_tensor, output_size)
    pred = pr_mask.softmax(dim=1).argmax(dim=1).squeeze(0).cpu().detach().numpy()
    mask = pred * 255
#     visualize(
#         img=mask
#     )
    cv2.imwrite(fn3%fid, mask)

In [8]:
# Aug 23 Generate resized video

x_pascal1200_dir = '/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/'
y_pascal1200_dir = '/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/'

# set up save dir
fn2 = r'/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F%05d.png'
fn3 = r'/playpen1/qiuyang/refined_segmentation/resized_original_image_1294/F%05d.png'
for fid in range(0, 1295):
    print(fn2%fid)
    image_vis = cv2.imread(fn2%fid)
    resize = albu.Compose([albu.Resize(height=612, width=512)])
    sample = resize(image=image_vis)
    image_vis = sample['image']

    cv2.imwrite(fn3%fid, image_vis)

/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00000.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00001.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00002.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00003.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00004.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00005.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00006.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00007.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00008.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/H

/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00080.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00081.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00082.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00083.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00084.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00085.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00086.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00087.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00088.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/H

/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00160.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00161.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00162.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00163.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00164.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00165.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00166.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00167.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00168.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/H

/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00240.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00241.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00242.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00243.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00244.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00245.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00246.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00247.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00248.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/H

/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00320.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00321.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00322.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00323.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00324.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00325.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00326.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00327.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00328.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/H

...

/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00960.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00961.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00962.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00963.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00964.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00965.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00966.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00967.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F00968.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/H

/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01040.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01041.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01042.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01043.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01044.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01045.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01046.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01047.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01048.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/H

/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01120.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01121.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01122.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01123.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01124.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01125.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01126.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01127.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01128.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/H

/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01200.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01201.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01202.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01203.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01204.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01205.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01206.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01207.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01208.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/H

/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01280.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01281.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01282.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01283.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01284.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01285.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01286.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01287.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/Henry_Lab/export/Teli_5_undistort_rot90/F01288.png
/playpen1/qiuyang/refined_segmentation/gdrive/MyDrive/H

In [5]:
# Aug 30 Generate resized Teli_7 video

x_dir = '/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/'
y_dir = '/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/'

# set up save dir
fn2 = r'/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F%05d_Teli_7.png'
fn3 = r'/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/resized_Teli_7_img/F%05d.png'
for fid in range(0, 628):
    print(fn2%fid)
    image_vis = cv2.imread(fn2%fid)
    resize = albu.Compose([albu.Resize(height=612, width=512)])
    sample = resize(image=image_vis)
    image_vis = sample['image']

    cv2.imwrite(fn3%fid, image_vis)

/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00000_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00001_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00002_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00003_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00004_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00005_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00006_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00007_Teli_7.png
/playpen1/qiuyang/refine

/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00068_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00069_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00070_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00071_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00072_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00073_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00074_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00075_Teli_7.png
/playpen1/qiuyang/refine

/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00136_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00137_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00138_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00139_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00140_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00141_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00142_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00143_Teli_7.png
/playpen1/qiuyang/refine

/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00204_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00205_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00206_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00207_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00208_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00209_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00210_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00211_Teli_7.png
/playpen1/qiuyang/refine

/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00272_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00273_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00274_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00275_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00276_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00277_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00278_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00279_Teli_7.png
/playpen1/qiuyang/refine

/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00340_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00341_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00342_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00343_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00344_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00345_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00346_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00347_Teli_7.png
/playpen1/qiuyang/refine

/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00408_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00409_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00410_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00411_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00412_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00413_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00414_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00415_Teli_7.png
/playpen1/qiuyang/refine

/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00476_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00477_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00478_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00479_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00480_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00481_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00482_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00483_Teli_7.png
/playpen1/qiuyang/refine

/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00544_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00545_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00546_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00547_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00548_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00549_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00550_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00551_Teli_7.png
/playpen1/qiuyang/refine

/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00612_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00613_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00614_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00615_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00616_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00617_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00618_Teli_7.png
/playpen1/qiuyang/refined_segmentation/teli_video/Teli_7_img/gdrive/MyDrive/Henry_Lab/Aug_30/Teli_7_dir/F00619_Teli_7.png
/playpen1/qiuyang/refine

In [ ]:
%rm -rf .ipynb_checkpoints/

In [ ]:
%ls -a

In [ ]:
%pwd

In [ ]:
%cd /playpen1/qiuyang/refined_segmentation/July_20_selfie/